In [1]:
import requests #for calling the API
import json #for de-serializing the response from the API
from IPython.display import HTML, display, YouTubeVideo #for displays, displaying YouTube Videos, and text formatting
import textwrap #for text formatting

In [10]:
def similar_movie(title, number): #calls TasteDive API
    '''Calls TasteDive API. Requires your own API key to run. Also, it needs the movie title and number of recommendations
    wanted as inputs.'''
    try: #attept
        url = "https://tastedive.com/api/similar" #base url
        with open("env_variables") as f: #open env_variables to read API key (file was on .gitignore)
            api_key = f.readlines()
            api_key = api_key[3]
            api_key = api_key[:-1]
        data = {'q': title, 'type': 'movies', 'info': [1], 'limit': [10], 'k': api_key} #parameters for API request
        resp = requests.get(url, data) #request the API with a GET
        rec = resp.json() #de-serializes the response
        count = 0 #initailize count
        while count <= (number - 1): #start poor-man's for loop
            try: #attempt to show the title of the film
                display(HTML(f"<h4>{rec['Similar']['Results'][count]['Name']}</h4>")) 
            except IndexError: #unless it's not there
                print(f"\nNo results found for '{title}'. \n") #then present error and set variables to null
                teaser = ''
                trailer = ''
                break
            try: #attempt to show the teaser and YouTube Video
                display(HTML(f"{textwrap.fill(rec['Similar']['Results'][count]['wTeaser'])}"))
                display(YouTubeVideo(rec['Similar']['Results'][count]['yID'], width=400, height=300))
            except KeyError: #unless it doesn't have any results
                print(f"\nNo results found for '{title}'. \n") #then present error and set variables to null
                teaser = ''
                trailer = ''
                break
            count += 1 #increase count for loop purposes
        #exception handling of connection errors when connecting to the API
    except requests.exceptions.RequestException as e:
        print("ERROR: Cannot connect to website, please try again later. ")
        print("DETAILS:", e)
    except requests.exceptions.HTTPError as e:
        print("ERROR: Response from the website was not ok.")
        print("DETAILS:", e)          
    except json.decoder.JSONDecodeError as e: 
        print("ERROR: Cannot decode the response with json")
        print("DETAILS", e)
        
    return #exit function



def query_movie(): #main logic for the Recommend branch of the program
    search = input("What film would you like to view recommendations from? Please enter the title: ")
    while True:
        number = input("How many recommendations would you like to view? Please enter a number, 1-10: ")
        try:
            number = int(number)
            if 1 <= number <= 10:
                break
            else:
                print("\nPlease select a numeral between 1 and 10. ")
        except ValueError:
            print("\nPlease enter a your selection as a numeral. ")                 
    similar_movie(search, number)
    menu_route = similar_continuation()
    return menu_route
                         
                         
def similar_continuation():
    while True:
        print("Would you like to: ")
        print("    Search for a different title? Enter '1' to search again. ")
        print("    Return to the main menu? Enter '2' to continue. ")
        print("    Quit. Enter 'q' to quit. \n")
        choice = input()
        if choice == '1':
            menu_route = False
            query_movie()
            break
        elif choice == '2':
            menu_route = True
            break
        elif choice == 'q':
            menu_route = False
            break
        else:
            print("Please make a valid selection. Choose '1' to search again, '2' to return, or 'q' to quit. ")
    return menu_route